# US-Ignite San Diego ETL Notebook

In [8]:
#import libraries for carto.
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
from cartoframes.viz.helpers import size_continuous_layer

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# import CityIq and libraries for timing
from cityiq import CityIq
import time
import json

#load .env with credentials
from dotenv import load_dotenv
load_dotenv()

import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)
# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()


## Extract: Bring Data into Notebook

In [12]:
# set time frame for use when querying for events (epoch time in milliseconds)
endTime = int(time.time())*1000 # time when demo.py is run
startTime = endTime-3600000 # startTime is 1 hour before endTime

# getting assets - assets with Pedestrian events
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT")
san_diego_pedestrian_sensor_metadata = myCIQ.getAssets()

# getting assets - assets with Temperature events
myCIQ.fetchMetadata("assets","environment","eventTypes:TEMPERATURE")
san_diego_environment_sensor_metadata = myCIQ.getAssets()

# getting assets - assets with Traffic events
myCIQ.fetchMetadata("assets","traffic","eventTypes:TFEVT")
san_diego_traffic_sensor_metadata = myCIQ.getAssets()

# getting assets - assets with Parking events
myCIQ.fetchMetadata("assets","parking","eventTypes:PKIN")
san_diego_parking_sensor_metadata = myCIQ.getAssets()

# getting assets - assets with Bicycle events
#myCIQ.fetchMetadata("assets","bicycle","eventTypes:BICYCLE")
#san_diego_bicycle_sensor_metadata = myCIQ.getAssets()






More Metadata assets are available for this query.
TotalElements: 437. Your specified size: 100
More Metadata assets are available for this query.
TotalElements: 338. Your specified size: 100
More Metadata assets are available for this query.
TotalElements: 520. Your specified size: 100


## Transform
Add here some code to do data transformation (remove outliers, null values, etc.). It's probably best to do this type of work on the geopandas dataframe 

In [13]:
san_diego_pedestrian_sensor_metadata_df = pd.DataFrame(san_diego_pedestrian_sensor_metadata)
san_diego_pedestrian_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_pedestrian_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_pedestrian_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_pedestrian_sensor_metadata_df["longitude"]= latlng[1].astype(float)
#print(san_diego_pedestrian_sensor_metadata_df)

san_diego_pedestrian_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_metadata_df.longitude, san_diego_pedestrian_sensor_metadata_df.latitude))
print(san_diego_pedestrian_sensor_metadata_gdf)

san_diego_pedestrian_sensor_metadata_d = Dataset(san_diego_pedestrian_sensor_metadata_gdf)
san_diego_pedestrian_sensor_metadata_d.upload(table_name='san_diego_pedestrian_sensor_metadata', if_exists='replace', credentials=credentials)


   assetType                              assetUid  \
0     CAMERA  1b576c8d-2b0c-4e44-991d-98dadc839bcc   
1     CAMERA  33ac5ea9-38e0-41f1-b7d1-a8af745a0f4a   
2     CAMERA  56f7e069-3b22-4195-8b82-865af18ce875   
3     CAMERA  638d378d-d288-4d59-adf0-469ad24bde57   
4     CAMERA  6b00875c-68f6-4d2e-b48c-f98833d619a5   
5     CAMERA  8f8da268-832a-4bff-bb66-d0ce4e5341f4   
6     CAMERA  9bad7b5a-d3b3-4578-9a2a-80e7b481aa72   
7     CAMERA  a813bcaf-a2e7-4b1a-8b4a-0f1f4171beef   
8     CAMERA  b25e8329-2bb1-4cd0-b949-533721bb5c1f   
9     CAMERA  d2c93e9b-e077-458f-adca-55cc0285347d   
10    CAMERA  d53dabee-fbc3-46b2-a41b-32ab1b283adc   
11    CAMERA  e3ff8b26-6e7a-4343-a9ea-f3bbe5efeb9d   
12    CAMERA  e7ff7742-714a-4e47-8f9a-997621d0adcc   
13    CAMERA  f2040f73-afbf-483c-a004-2260a4828f60   
14    CAMERA  fd434a3e-af3a-4ee8-bd75-f2c38b537fdb   

                           coordinates                             eventTypes  \
0   32.7095162804853:-117.156174617791  [PKIN, TFEVT, 

In [14]:
san_diego_traffic_sensor_metadata_df = pd.DataFrame(san_diego_traffic_sensor_metadata)
san_diego_traffic_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = san_diego_traffic_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 

san_diego_traffic_sensor_metadata_df["latitude"]= latlng[0].astype(float)
san_diego_traffic_sensor_metadata_df["longitude"]= latlng[1].astype(float)
#print(san_diego_traffic_sensor_metadata_df)

san_diego_traffic_sensor_metadata_gdf = gpd.GeoDataFrame(san_diego_traffic_sensor_metadata_df, geometry=gpd.points_from_xy(san_diego_traffic_sensor_metadata_df.longitude, san_diego_traffic_sensor_metadata_df.latitude))
print(san_diego_traffic_sensor_metadata_gdf)



   assetType                              assetUid  \
0     CAMERA  000223ee-a868-474b-abcb-12ff1bad00a3   
1     CAMERA  00c6b5d6-df99-4696-be16-d274af05ea89   
2     CAMERA  0161240c-aa9c-4a60-b693-d7af5270ebe3   
3     CAMERA  02276cc4-dc67-4e12-ab0c-ffe3d19747c4   
4     CAMERA  02fd833c-5d93-45f3-99fc-844c492f3067   
..       ...                                   ...   
95    CAMERA  43d86e46-b953-4b48-8da3-464dc28e09da   
96    CAMERA  456bd0cc-0fd6-40d8-8b48-020ff22023b6   
97    CAMERA  46b9df1d-cd09-4ae4-8d54-b06d8bb7566f   
98    CAMERA  46be7cfa-2f64-4485-a24f-9e3e5eeba636   
99    CAMERA  4758de5b-a8f3-46e4-a022-04ea72cf114c   

                           coordinates                     eventTypes  \
0             32.71143062:-117.1600173  [PKOUT, PKIN, TFEVT, BICYCLE]   
1             32.71268097:-117.1543763  [PKOUT, TFEVT, PKIN, BICYCLE]   
2             32.71351413:-117.1561822  [PKOUT, PKIN, BICYCLE, TFEVT]   
3   32.7154760627819:-117.152087034359  [PKIN, PKOUT, TFEVT

In [15]:
san_diego_pedestrian_sensor_metadata_gdf.columns

Index(['assetType', 'assetUid', 'coordinates', 'eventTypes', 'mediaType',
       'parentAssetUid', 'latitude', 'longitude', 'geometry'],
      dtype='object')

## Load: Upload to CARTO

In [6]:
san_diego_pedestrian_sensor_metadata_d = Dataset(san_diego_pedestrian_sensor_metadata_gdf)
san_diego_pedestrian_sensor_metadata_d.upload(table_name='san_diego_pedestrian_sensor_metadata', if_exists='replace', credentials=credentials)

The following columns were changed in the CARTO copy of this dataframe:
assetType -> assettype
assetUid -> assetuid
eventTypes -> eventtypes
mediaType -> mediatype
parentAssetUid -> parentassetuid


/home/pui-user/anaconda3/lib/python3.6/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 62389b7d-37df-4886-afac-247339c477c0
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


In [16]:
san_diego_traffic_sensor_metadata_d = Dataset(san_diego_traffic_sensor_metadata_gdf)
san_diego_traffic_sensor_metadata_d.upload(table_name='san_diego_traffic_sensor_metadata', if_exists='replace', credentials=credentials)

The following columns were changed in the CARTO copy of this dataframe:
assetType -> assettype
assetUid -> assetuid
eventTypes -> eventtypes
mediaType -> mediatype
parentAssetUid -> parentassetuid


/home/pui-user/anaconda3/lib/python3.6/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: b84b9c5a-aa1d-48cb-a4ac-45f6cb5735cd
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


## ETL for loading Events

In [18]:
# empty list to collect events
san_diego_pedestrian_sensor_events_list = []
# empty list of aggregates
san_diego_pedestrian_sensor_aggregate_list = []
# loop through assets to fetch events for each asset
for index, row in san_diego_pedestrian_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "PEDEVT", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        a["directionUnit"] = a["properties"]["directionUnit"]
        a["speedUnit"] = a["properties"]["speedUnit"]
        a["eventUid"] = a["properties"]["eventUid"]
        a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
        a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
        a["pedestrianCount"] = a["measures"]["pedestrianCount"]
        a["counter_direction"] = a["measures"]["counter_direction"]
        a["speed"] = a["measures"]["speed"]
        a["direction"] = a["measures"]["direction"]
        san_diego_pedestrian_sensor_events_list.append(a)
    
print(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df = pd.DataFrame(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df.dropna(inplace = True)

# group by location ID to get a sum of pedestrianCounts 
grouped_SD_ped_sensor_events_df = san_diego_pedestrian_sensor_events_df.groupby('assetUid').agg({'pedestrianCount': ['sum']})
grouped_SD_ped_sensor_events_df.columns = ['pedestrianCount_sum']
grouped_SD_ped_sensor_events_df.reset_index()
# print(grouped_SD_ped_sensor_events_df)

# join counts back to sensor metadata
merged_metadata_ped_counts_df = pd.merge(san_diego_pedestrian_sensor_metadata_df, grouped_SD_ped_sensor_events_df, on=['assetUid'])
# print(merged_metadata_ped_counts_df)

merged_metadata_ped_counts_gdf = gpd.GeoDataFrame(merged_metadata_ped_counts_df, geometry=gpd.points_from_xy(merged_metadata_ped_counts_df.longitude, merged_metadata_ped_counts_df.latitude))

merged_metadata_ped_counts_d = Dataset(merged_metadata_ped_counts_gdf)
merged_metadata_ped_counts_d.upload(table_name='merged_metadata_ped_counts_san_diego', if_exists='replace', credentials=credentials)

    
#san_diego_pedestrian_sensor_events_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_events_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_events_df.longitude, san_diego_pedestrian_sensor_events_df.latitude))
#san_diego_pedestrian_sensor_events_d = Dataset(san_diego_pedestrian_sensor_events_gdf)
#san_diego_pedestrian_sensor_events_d.upload(table_name='san_diego_pedestrian_sensor_events', if_exists='append', credentials=credentials)

[{'locationUid': 'kvmzs7jn8srk0sm5ta4', 'assetUid': '1b576c8d-2b0c-4e44-991d-98dadc839bcc', 'eventType': 'PEDEVT', 'timestamp': 1570376558168, 'properties': {'directionUnit': 'DEGREE', 'speedUnit': 'METERS_PER_SEC', 'eventUid': 'MTU3MDM3NjU1ODE2OA=='}, 'measures': {'counter_direction_speed': 1.8904975258856256, 'counter_direction_pedestrianCount': 2.0, 'pedestrianCount': 0.0, 'counter_direction': 94.0, 'speed': 0.0, 'direction': 274.0}, 'latitude': 32.7095162804853, 'longitude': -117.156174617791, 'directionUnit': 'DEGREE', 'speedUnit': 'METERS_PER_SEC', 'eventUid': 'MTU3MDM3NjU1ODE2OA==', 'counter_direction_speed': 1.8904975258856256, 'counter_direction_pedestrianCount': 2.0, 'pedestrianCount': 0.0, 'counter_direction': 94.0, 'speed': 0.0, 'direction': 274.0}, {'locationUid': 'kvmzs7jn8srk0sm5ta4', 'assetUid': '1b576c8d-2b0c-4e44-991d-98dadc839bcc', 'eventType': 'PEDEVT', 'timestamp': 1570376588226, 'properties': {'directionUnit': 'DEGREE', 'speedUnit': 'METERS_PER_SEC', 'eventUid': '

The following columns were changed in the CARTO copy of this dataframe:
assetType -> assettype
assetUid -> assetuid
eventTypes -> eventtypes
mediaType -> mediatype
parentAssetUid -> parentassetuid
pedestrianCount_sum -> pedestriancount_sum


/home/pui-user/anaconda3/lib/python3.6/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 52677fa8-acc2-47a8-8b81-9120ebdc0a53
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


In [22]:
# empty list to collect events
san_diego_traffic_sensor_events_list = []
# empty list of aggregates
san_diego_traffi_sensor_aggregate_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_traffic_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "TFEVT", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
   


In [28]:
print(san_diego_traffic_sensor_events_list)

[]


In [24]:
# empty list to collect events
san_diego_traffic_sensor_events_list = []
# empty list of aggregates
san_diego_traffi_sensor_aggregate_list = []


# loop through assets to fetch events for each asset
for index, row in san_diego_traffic_sensor_metadata_gdf.iterrows():
    myCIQ.fetchEvents("assets", row.assetUid, "TRAFEVT", startTime, endTime, pageSize=500)
    assetEvents = myCIQ.getEvents()
    
print(san_diego_pedestrian_sensor_events_list)
    for a in assetEvents:
        a["latitude"] = row.latitude
        a["longitude"] = row.longitude
        a["directionUnit"] = a["properties"]["directionUnit"]
        a["speedUnit"] = a["properties"]["speedUnit"]
        a["eventUid"] = a["properties"]["eventUid"]
        a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
        a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
        a["pedestrianCount"] = a["measures"]["pedestrianCount"]
        a["counter_direction"] = a["measures"]["counter_direction"]
        a["speed"] = a["measures"]["speed"]
        a["direction"] = a["measures"]["direction"]
        san_diego_pedestrian_sensor_events_list.append(a)
    
print(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df = pd.DataFrame(san_diego_pedestrian_sensor_events_list)
san_diego_pedestrian_sensor_events_df.dropna(inplace = True)
# print(san_diego_pedestrian_sensor_events_df)

# group by location ID to get a sum of pedestrianCounts 
grouped_SD_ped_sensor_events_df = san_diego_pedestrian_sensor_events_df.groupby('assetUid').agg({'pedestrianCount': ['sum']})
grouped_SD_ped_sensor_events_df.columns = ['pedestrianCount_sum']
grouped_SD_ped_sensor_events_df.reset_index()
# print(grouped_SD_ped_sensor_events_df)

# join counts back to sensor metadata
merged_metadata_ped_counts_df = pd.merge(san_diego_pedestrian_sensor_metadata_df, grouped_SD_ped_sensor_events_df, on=['assetUid'])
# print(merged_metadata_ped_counts_df)

merged_metadata_ped_counts_gdf = gpd.GeoDataFrame(merged_metadata_ped_counts_df, geometry=gpd.points_from_xy(merged_metadata_ped_counts_df.longitude, merged_metadata_ped_counts_df.latitude))

merged_metadata_ped_counts_d = Dataset(merged_metadata_ped_counts_gdf)
merged_metadata_ped_counts_d.upload(table_name='merged_metadata_ped_counts_san_diego', if_exists='replace', credentials=credentials)

    
#san_diego_pedestrian_sensor_events_gdf = gpd.GeoDataFrame(san_diego_pedestrian_sensor_events_df, geometry=gpd.points_from_xy(san_diego_pedestrian_sensor_events_df.longitude, san_diego_pedestrian_sensor_events_df.latitude))
#san_diego_pedestrian_sensor_events_d = Dataset(san_diego_pedestrian_sensor_events_gdf)
#san_diego_pedestrian_sensor_events_d.upload(table_name='san_diego_pedestrian_sensor_events', if_exists='append', credentials=credentials)

IndentationError: unexpected indent (<ipython-input-24-cc32c100bc4f>, line 13)

In [3]:
# set time frame for use when querying for events (epoch time in milliseconds)
endTime = int(time.time())*1000 # time when demo.py is run
startTime = endTime-3600000 # startTime is 1 hour before endTime

# get CityIq token
myCIQ = CityIq("City")
myCIQ.fetchToken()

# getting assets - assets with PEDEVT events
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT")
san_diego_pedestrian_sensor_metadata = myCIQ.getAssets()

## Make a quick map using one of CARTO's helper functions

In [12]:
Map(
    size_continuous_layer(merged_metadata_ped_counts_d, 'pedestrianCount_sum', 'Pedestrian Count')
)